Dataset From Kaggle 
https://www.kaggle.com/datasets/aslanahmedov/market-basket-analysis?resource=download
Market Basket Analysis

In [ ]:
from Shortcuts import ToolBox
import pandas as pd
import numpy as np

# Data visualisierung
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker

import pandas as pd
import plotly.express as px
import plotly.figure_factory as ff

In [ ]:
# Load the dataset
data_raw = pd.read_csv('Assignment-1_Data.csv', sep=';')
data_raw.head()

In [ ]:
## take a lot about our branches countries
data_raw['Country'].unique()

In [ ]:
# user defined function to show missing values, describe the data stats, and a sample from the data
ToolBox.explore_df(data_raw)

## Data Problems


- Negative quantites
- Date Syntax
- Missing Values
- Price Column include commas
- Remove outliers

Using this Dataset is very valuable for me to test my skills in Analyising the market basket trends and sales and identify pattern as in an industry of retails as that's my most important interest in the real world problems

## Data Cleaning 

In [ ]:
# Converting Date Column
data_raw['Date'] = pd.to_datetime(data_raw['Date'], format='%d.%m.%Y %H:%M')
data_raw['YearMonth'] = data_raw['Date'].dt.to_period('M')
display(data_raw)

In [ ]:
# We notice that 'Price' column has commas in the numeric values, let's replace them and convert it to float
data_raw['Price'] = data_raw['Price'].str.replace(',', '.').astype(float)
display(data_raw)

In [ ]:
print("percentage of missing data ")
print(data_raw.isnull().sum() / data_raw.shape[0]*100)
print(data_raw.shape[0])

A crucial aspect of data analysis is ensuring the data's integrity by filtering out irrelevant or erroneous entries and handling missing values appropriately. In this chapter, we focus on filtering out non-positive values, removing rows with missing item names, filling in missing customer IDs, and calculating total prices per transaction. These steps are vital to ensure the accuracy and reliability of our subsequent analysis.

As the missing data is considered a sensitive data ( item name and customer ID) we can't fill it with any of the recommened rechiniques as we could do with price, qunatity or even the date
- We will drop the missing values for the items 
- Instead of dropping missing values because of the customers IDs we will fill them with thresholder 'NA'
- We will filter out all negative prices and quantities as a wrong data entry bug

In [ ]:
# Filtering the dataset to include only rows with values > 0
data = data_raw[(data_raw['Quantity']>0) & (data_raw['Price']>0)]

In [ ]:
# Dropping rows without item 
data = data[data['Itemname'].notnull()]

In [ ]:
# Filling missing customer IDs
data = data.fillna('#NA')

In [ ]:
# cleaning item names column
data['Itemname'] = data['Itemname'].str.lower()
data['Itemname'] = data['Itemname'].str.strip()

#### Adding Attributes

In [ ]:
# Calculate GMV per pos / transaction
# GMV -> Gross Margin Value 
data['GMV'] = data['Quantity'] * data['Price']

#### Cleaning outliers

In [ ]:
# Create box plots using Plotly Express
fig = px.box(data, y=['Quantity', 'GMV'], 
             title='Box Plot of Quantity and GMV (Outliers Removed)',
             labels={'variable': 'Attribute', 'value': 'Value'})

# Show the plot
fig.show()

In [ ]:
# Remove outliers from 'Quantity' column
data_cleaned = ToolBox.remove_outliers_iqr(data, 'Quantity')

# Remove outliers from 'GMV' column
data_cleaned = ToolBox.remove_outliers_iqr(data, 'GMV')

We dropped almost 10% of the values as they are highly inflated and skewed

### Investigations Aspects
- What's the sales monthly Trend All over the world?
- Which is the most important countries with respect to the achieved sales?
- what's the most sold item in each country?
- in each month which country is bringing most of our revenues?
- Relation between Sales and quantity sold?
- Customer Behavior?

## Exploratory Data Analysis

1- What's the sales monthly Trend All over the world?
- We group the data by month and year, calculating the total sum of sales to understand the sales trend over time. The resulting visualization depicts the total sales per month.

In [ ]:
# Convert 'YearMonth' column to string format
data_cleaned['YearMonth'] = data_cleaned['YearMonth'].astype(str)

# Grouping the data by month and year, and calculating the total sum of sales
monthly_sales = data_cleaned.groupby('YearMonth')['GMV'].sum().reset_index()

# Calculate the average GMV across all months
average_gmv = monthly_sales['GMV'].mean()

# Create a line chart using Plotly Express
fig = px.line(monthly_sales, x='YearMonth', y='GMV', title='Total GMV by Month', 
              labels={'YearMonth': 'Year-Month', 'GMV': 'Total GMV'})

# Add a line for the average GMV
fig.add_hline(y=average_gmv, line_dash="dot", line_color="red", annotation_text=f'Average GMV ({average_gmv:.2f})', 
              annotation_position="bottom right")

# Show the plot
fig.show()

We can see that our sales increased dramatically from Aug until reached the peak on Nov and then dropped down below the average by in DEC

2- Which is the most important countries with respect to the achieved sales?


In [ ]:
# Group by 'Country' and calculate the sum of 'GMV'
sales_by_country = data_cleaned.groupby('Country')['GMV'].sum().reset_index()

# Sort the DataFrame by 'GMV' in descending order
sales_by_country = sales_by_country.sort_values(by='GMV', ascending=False)

# Create a bar chart using Plotly Express
fig = px.bar(sales_by_country, x='Country', y='GMV', 
             title='Total Sales by Country',
             labels={'Country': 'Country', 'GMV': 'Total Sales'})

# Show the plot
fig.show()

We can conclude that United Kingdom is by far the main stream of sales to our market which needs to studied as a case to apply on the rest of the countries market to be able to develope our market shares

3-  what's the most sold item in each country?


In [ ]:
# Group by 'Country' and 'Itemname' and calculate the sum of 'GMV'
sales_by_country_item = data_cleaned.groupby(['Country', 'Itemname'])['GMV'].sum().reset_index()

# Find the index of the maximum GMV for each country
idx = sales_by_country_item.groupby('Country')['GMV'].idxmax()

# Get the most sold item in each country
most_sold_items = sales_by_country_item.loc[idx]

# Sort the DataFrame by 'GMV' in descending order
most_sold_items = most_sold_items.sort_values(by='GMV', ascending=False)

# Create a bar chart using Plotly Express
fig = px.bar(most_sold_items, x='Country', y='GMV', color='Itemname',
             title='Most Sold Item in Each Country by GMV',
             labels={'Country': 'Country', 'GMV': 'Total GMV', 'Itemname': 'Most Sold Item'})

# Show the plot
fig.show()

Now we got an insight about the main product we are marketing in each country, we need to focus our compaigns on these products and search for growth opportunity

4- in each month which country is bringing most of our revenues?


In [ ]:
# Group by 'YearMonth' and 'Country' and calculate the sum of 'GMV'
revenue_by_month_country = data_cleaned.groupby(['YearMonth', 'Country'])['GMV'].sum().reset_index()

revenue_by_month_country_filtered = revenue_by_month_country.query("Country != 'United Kingdom' &  Country != 'Israel'")


# Calculate the total GMV for each month
total_gmv_by_month = revenue_by_month_country_filtered.groupby('YearMonth')['GMV'].transform('sum')

# Calculate the percentage contribution of each country to the total sales for each month
revenue_by_month_country_filtered['Contribution (%)'] = (revenue_by_month_country_filtered['GMV'] / total_gmv_by_month) * 100

# Display the result
print(revenue_by_month_country_filtered)

As it's a huge amount of values and countries it would be better to be investigated on an operational level to dig deeper on the countries contributions on my sales on a monthly base

5- Relation between Sales and quantity sold?


In [ ]:
# Create a scatter plot using Plotly Express
fig = px.scatter(data_cleaned, x='Quantity', y='GMV', 
                 title='Relation between Sales and Quantity Sold',
                 labels={'Quantity': 'Quantity Sold', 'GMV': 'Sales'})

# Show the plot
fig.show()

### Unique Item Analysis

Next, we analyze the uniqueness of items sold per month by grouping the data and calculating the count of unique items. The line plot visualizes the sum of unique items per month.

In [ ]:
# Grouping the data by month and year, and calculating the unique items per month
monthly_item = data.groupby('YearMonth')['Itemname'].nunique()
plt.figure(figsize=(15,5))
monthly_item.plot(kind='line', marker='o', color='b')
plt.title('Sum of Unique Items per Month')
plt.xlabel('Month')
plt.ylabel('Sum of Items')
plt.grid(True)
plt.show()